> ![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/98/International_Pok%C3%A9mon_logo.svg/2000px-International_Pok%C3%A9mon_logo.svg.png)


> **Pokémon is short for the original Japanese title of Pocket Monsters. **

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns  
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#importing all the required ML packages
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from mpl_toolkits.mplot3d import Axes3D
# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# word cloud library
from wordcloud import WordCloud
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/Pokemon.csv", index_col=0)
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data.shape

> 
>As we see there are 721 pokemon. However 800 rows are in data set. That is because some of the pokemons have versions with different feature values because of evolving in different levels.
>
>For instance, 
>Venusaur is a dual-type Grass/Poison Pokémon introduced in Generation I. 
>It evolves from Ivysaur starting at level 32. It is the final form of Bulbasaur. It can Mega Evolve into Mega Venusaur using the Venusaurite.
>
>These pairs are indexed with same number

In [ ]:
data.columns

Let's rename the features to not have problem later.

In [ ]:
 
data = data.rename(columns={"#":"index", "Type 1":"Type1","Type 2":"Type2",
                           "HP":"HP", "Sp. Atk": "Special_Attack",
                           "Sp. Def":"Special_Defense"})
data.info()

In [ ]:
#some values in TYPE2 are empty and thus they have to be filled or deleted
data['Type2'].fillna(data['Type1'], inplace=True) 
#fill NaN values in Type2 with corresponding values of Type
data.head()

In [ ]:
#let see the correlation between features:

f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(data.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax ,cmap="Blues")
plt.show()


> There isn't so much correlated feaures that can be considered important.
>
> 'Special Attack' and 'Special Defense' is correlated with being legendary with 0.4 ratio
> 'Speed' and 'Attack' are correlated with 0.5 ratio.
>
> The highest we can see is the correlation between Sp.Atk and the Total

**Let's see the distribution of the HP, Attack, Defense, Special Attack, Special Defense, and Speed, Total stats.**

In [ ]:
fig, axarr = plt.subplots(3, 2, figsize=(13, 13))
#ax = plt.subplot2grid((3,2),(0,0), rowspan=1, colspan=1, fig=None) also could be used

# Histogram of HP
sns.distplot(data['HP'], hist=True, kde=False, bins=50, color = 'blue', 
             hist_kws={'edgecolor':'black'}, ax=axarr[0][0], axlabel='HP')

# Histogram of Attack
sns.distplot(data['Attack'], hist=True, kde=False, bins=50, color = 'blue', 
             hist_kws={'edgecolor':'black'},ax=axarr[0][1], axlabel='Attack' )

# Histogram of Defense
sns.distplot(data['Defense'], hist=True, kde=False, bins=50, color = 'blue', 
             hist_kws={'edgecolor':'black'}, ax=axarr[1][0], axlabel='Defense')

# Histogram of Special Attack
sns.distplot(data['Special_Attack'], hist=True, kde=False, bins=50, color = 'blue', 
             hist_kws={'edgecolor':'black'}, ax=axarr[1][1], axlabel='Special Attack')

#Histogram of Special Defense
sns.distplot(data['Special_Defense'], hist=True, kde=False, bins=50, color = 'blue', 
             hist_kws={'edgecolor':'black'}, ax=axarr[2][0], axlabel='Special Defense')

# Histogram of Speed
sns.distplot(data['Speed'], hist=True, kde=False, bins=50, color = 'blue', 
             hist_kws={'edgecolor':'black'}, ax=axarr[2][1], axlabel='Speed')

plt.show()

In [ ]:
# Histogram of Total

sns.distplot(data['Total'], hist=True, kde=False, bins=50, color = 'purple', 
             hist_kws={'edgecolor':'black'}, axlabel='Total')
plt.axvline(data['Total'].mean(),linestyle='dashed',color='yellow') #line on average of Total

figsize=(5, 5)

plt.show()

> As it is expected;
>
> 
>
>*  Defence & Special Defence distrubution are quite similar.
 >
>* Speed distrubution is close to Special Attack and Attack distributions.

> **let's see Pokémon's combat stats in a box plot:**

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize']=10,10  #to adjust the plot size

df = data.drop(['Total', 'Generation', 'Legendary'], axis=1)
sns.boxplot(data=df) 

plt.show()

>-------------------- 
>
>Let's see the relation of Defense & Attack by seing them in different plots.

In [ ]:
# Density Plot
sns.kdeplot(data.Attack, data.Defense)
plt.show()

Interactive Line Charts:

In [ ]:
#prepare data frame
Ndf = data.iloc[:50,:]

# Creating trace1
trace1 = go.Scatter(y = Ndf.Attack,
                    x = Ndf.index,
                    mode = "lines+markers",
                    name = "Attack",
                    marker = dict(color = 'blue'))
# Creating trace2
trace2 = go.Scatter(y = Ndf.Defense,
                    x = Ndf.index,
                    mode = "lines",
                    name = "Defence",
                    marker = dict(color = 'red'))

dataS = [trace1, trace2]
layout = dict(title = 'Attack and Defense of the pokemons',
              xaxis= dict(title= 'Index',ticklen= 5,zeroline= False)
             )
fig = dict(data = dataS, layout = None)
iplot(fig)

>------------------ 
>Wordcloud plot of Type1 pokemons in Generation1

In [ ]:
x2011 = data.Type1[data.Generation == 1]
plt.subplots(figsize=(8,8))
wordcloud = WordCloud(
                          background_color='white',
                          width=400,
                          height=300
                         ).generate(" ".join(x2011))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('graph.png')

plt.show()

>---------------------------
>
>3D Scatter Plot

In [ ]:
# create trace 1 that is 3d scatter
trace1 = go.Scatter3d(
                    y = Ndf.Attack,
                    x = Ndf.Defense,
                    z = Ndf.index,
                    mode = "markers",
                    name = "Attack",
                    marker=dict(size=10,color='rgb(255,0,0)'))

Data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0  
    )
    
)
fig = go.Figure(data=Data, layout=layout)

iplot(fig)

x stands for Defence
y for Attack 
z for Index number of the pokemon

In [ ]:
# Joint Distribution Plot
sns.jointplot(x='Attack', y='Defense', data=data)
plt.show()

**LET'S ANALYSE THE DATA BY TAKİNG "GENERATION" FEATURE INTO CONSIDERATION**

In [ ]:
#let's compare the Attack and Defense stats

sns.lmplot(x='Attack', y='Defense', data=data, size=4, aspect=2, fit_reg=True, 
           hue='Generation')
plt.show()


Linear regration line for Generation2 has biggest slope and lowest slope is of generation5 . We can say that the highest defense in the same point of attack is of Generation2

Generation 1 and 3 has similar Defence & Attack relation. 

In [ ]:
data.groupby(['Generation']).size().reset_index(name='counts')
#data.groupby(['Generation']).size()

In [ ]:
sns.set(style="darkgrid")
sns.set_context(font_scale =20)
sns.countplot(x='Generation',data=data,saturation=0.75,palette="Blues_d", hue='Legendary')
plt.xlabel('Generation', fontsize=15) 
plt.ylabel('Number', fontsize=15) 
plt.title('Number of the pokemons around different generations', fontsize=15)

plt.show()

Generation 3 has more Legendary Pokémon.





In [ ]:
DType=data.groupby(['Generation','Type1']).count().reset_index()
DType=DType[['Generation','Type1','Total']]
DType=DType.pivot('Generation','Type1','Total')
DType[['Water','Fire','Grass','Dragon','Normal','Rock','Flying','Electric']].plot(marker='*')
fig=plt.gcf()
fig.set_size_inches(7,5)

DType=data.groupby(['Generation','Type2']).count().reset_index()
DType=DType[['Generation','Type2','Total']]
DType=DType.pivot('Generation','Type2','Total')
DType[['Water','Fire','Grass','Dragon','Normal','Rock','Flying','Electric']].plot(marker='*')
fig=plt.gcf()
fig.set_size_inches(7,5)

plt.show()

In [ ]:
plt.subplots(figsize = (15,5))
plt.title('Strongest Genaration')
sns.violinplot(x='Generation',data=data, y = "Total")
plt.show()

>**RESULT **
>
>Pokemons had the highest numbers in the 1st Generation. However, the number has decreased in 2nd generation. It has increased again in 4th and 5th generation. At the end it has the lowest number in generation 6.
>
> Water pokemons has the highest numbers in the 1st Generation. However the number has decreased with passing generations. Similarly Grass type pokemons showed an increase in their numbers till generation 5.  The number of Type2 Grass Pokemons has been steadily increasing. The same is the case for the Dragon Type Pokemons.
>
> About Generations:
> As we can see from the plot and also know from the video games there are 6 generation of pokemons relased in different times.
> * Generation 1 includes Pokemon 1-151
> * Generation 2 includes Pokemon 152-251
> * Generation 3 includes Pokemon 252-386
> * Generation 4 includes Pokemon 387-493
> * Generation 5 includes Pokemon 494-649
> * Generation 6 includes Pokemon 650-721
>
>
> **And the last violinplot shows that generation 3 is the strongest generation.**
>
>...

**LET'S ANALYSE THE DATA BY TAKİNG "TYPE" 1 AND "TYPE2" FEATURES INTO CONSIDERATION**

In [ ]:
print('The unique  pokemon types are: ','\n',data['Type1'].unique(),'\n','\n') #unique types of column
print('The number of unique types are: ''\n',data['Type1'].nunique()) #count of unique values 


In [ ]:
gdf = data.groupby(by=['Type1', 'Type2']).agg(['max','min']).head(10)
gdf

In [ ]:
data.groupby(['Type1']).size().reset_index(name='counts')

In [ ]:
plt.subplots(figsize=(10,15))
ax = data['Type1'].value_counts().sort_values(ascending=True).plot.barh(width=.9,
                                                    color=sns.color_palette('inferno',40))
ax.set_xlabel('count')
ax.set_ylabel('types')
plt.title("Number of various pokemon/type1",loc='left', fontsize=16)

plt.show()

In [ ]:
data.groupby(['Type2']).size().reset_index(name='counts')

Let's see both distribution of various pokemon around Type1/Type2 in Pie Plot

In [ ]:
#Lets plot both together in 2 rows 1 cols
#first row, first col
ax1 = plt.subplot2grid((2,1),(0,0))
labels = 'Water', 'Normal', 'Grass', 'Bug', 'Psychic', 'Fire', 'Electric', 'Rock', 'Other'
sizes = [112, 98, 70, 69, 57, 52, 44, 44, 175]
explode = (0, 0, 0.0, 0, 0, 0, 0, 0, 0.1)  # explode the last slice 
plt.pie(sizes, explode=explode, labels=labels,autopct='%1.1f%%', shadow=True, startangle=90, 
        colors=sns.cubehelix_palette(8, start=.5, rot=-.75), 
        labeldistance=1.1, rotatelabels = False)

plt.axis('equal')
plt.title("Distribution of various pokemon/type1",loc='left', fontsize=16)
plt.plot()
fig=plt.gcf()
fig.set_size_inches(12,12)


#Second row first column
ax1 = plt.subplot2grid((2,1), (1, 0))

labels = 'Water', 'Normal', 'Grass', 'Bug', 'Psychic', 'Fire', 'Electric', 'Rock', 'Other'
sizes = [73, 65, 58, 20, 71, 40, 33, 23,286]
explode2 = (0, 0, 0, 0, 0.1, 0, 0, 0, 0)
plt.pie(sizes, labels=labels,explode=explode2,autopct='%1.1f%%', shadow=True, startangle=90, 
        colors=sns.cubehelix_palette(8, start=.5, rot=-.75), 
        labeldistance=1.1, rotatelabels = False)

plt.axis('equal')
plt.title("Distribution of various pokemon/type2",loc='right', fontsize=16)
plt.plot()
fig=plt.gcf()
fig.set_size_inches(12,12)
plt.show()

In [ ]:

plt.subplots(figsize = (15,3))
plt.title('Attack by Type1')
sns.violinplot(x = "Type1", y = "Attack",data = data)
plt.ylim(-100,250)

plt.subplots(figsize = (15,3))
plt.title('Attack by Type2')
sns.violinplot(x = "Type2", y = "Attack",data = data)
plt.ylim(-100,250)
plt.show()

Violinplot plots according to the density of a region. This means that the parts of the plot where the width is thicker denotes a region with higher density points whereas regions with thinner area show less densely populated points.
>-
>
**For Type 1:**
>
>It is shown that the Psychic type pokemons have an edge over the other types as they have a higher attacks compared to the other types. Also since the fire pokemons have lower range of values, but higher attacks, they can be preferred over the grass and water types for attacking.
>
**For Type2:**
>
> It is shown that the Psychic and Fighting type pokemons have an edge over the other types as they have a higher attacks compared to the other types. 
> Also since the rock pokemons have lower range of values, but higher attacks, they can be preferred over the grass and water types for attacking.

In [ ]:
# Melt DataFrame
#Let's melt all 6 of the stat columns into one
#The new Stat column indicates the original stats (HP, Attack, Defense, Sp. Attack...)
TD_data = data.drop(['Total', 'Generation','Legendary'], axis=1)
M_df = pd.melt(TD_data, 
                    id_vars=["Name", "Type1", "Type2"], # Variables to keep
                    var_name="Stat") # Name of melted variable
M_df.head()

In [ ]:
# M_df Swarmplot 
plt.figure(figsize=(15,5))
sns.swarmplot(x='Stat', y='value', data=M_df, hue='Type1', palette=sns.color_palette("coolwarm", 18)
              , split=True)
# 4. Adjust the y-axis
plt.ylim(0, 260)
# 5. Place legend to the right
plt.legend(bbox_to_anchor=(1, 1), loc=2)
plt.show()

While 'Steel' type of pokemons have bigger value of defence, 'Normal' has lowest values of Defense.
We can compare the stats of types by this plot easily.

In [ ]:
#Psychic Vs Poison
fire=data[(data['Type1']=='Psychic') | ((data['Type2'])=="Psychic")] #fire contains all fire pokemons
water=data[(data['Type1']=='Poison') | ((data['Type2'])=="Poison")]  #all water pokemins
plt.scatter(fire.Attack.head(50),fire.Defense.head(50),color='Y',label='Psychic',marker="*",s=50) 
plt.scatter(water.Attack.head(50),water.Defense.head(50),color='G',label="Poison",s=25)
plt.xlabel("Attack")
plt.ylabel("Defense")
plt.legend()
plt.plot()
fig=plt.gcf()  #get the current figure using .gcf()
fig.set_size_inches(12,6) #set the size for the figure
plt.show()

This shows that Psychic type pokemons have a better defence than Poison type pokemons but have lower attact.

>-----------------
>
>
>**PREDICTIVE MODELING** 
>
>
>We have gained some insights from the visualisation of data. Now let's see if we can accurately predict some of the features' values
>
>The steps to building and using a model are:
>
> * **Define:** What type of model will it be? 
> * **Fit:** Capture patterns from provided data. 
> * **Predict**: Use the model to predict.
> * **Evaluate:** Determine how accurate the model's predictions are.
>...

**Let's select data for modeling**

In [ ]:
#we'll build a model with only a some featuresthat are int type
#We select multiple features
#Speed is prediction Target = y
pokemon_features = ['HP', 'Attack', 'Defense', 
                    'Special_Attack', 'Special_Defense','Generation']
DatatoModel = data[pokemon_features].reindex()
DatatoModel.head()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
y = data['Speed']  #prediction target
# Define model. Specify a number for random_state to ensure same results each run
ModelData = DecisionTreeRegressor(random_state=1)

# Fit model
ModelData.fit(DatatoModel, y) 

>_______________________
>
#    We now have a fitted model that we can use to make predictions
>
>************

In [ ]:
print("Making predictions for the following 5 pokemon:")
print(DatatoModel.head())
print("The predictions are")
print(ModelData.predict(DatatoModel.head()))

>___________________________________________
>
>We used the model to the Speed values.
>
> We already have Speed values. But we made predictions for the first 5 rows of the training data to see how the predict function works and it worked perfectly.
>
>___________________________________________

>** LET'S TRY ANOTHER MODEL**

In [ ]:
data.groupby('Legendary').apply(np.mean)

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(data[data['Legendary']==False].Total, color="red", label="False" )
sns.distplot(data[data['Legendary']==True].Total, color="skyblue", label="True")
legend = [True, False]
plt.legend(bbox_to_anchor=(1, 1), loc=2)
plt.show()

In [ ]:
%matplotlib inline

sns.boxplot(x='Legendary', y='Total', data=data)
plt.show()

In [ ]:
data['Legendary'].value_counts()

In [ ]:
sns.countplot(x='Legendary', data = data, palette = 'hls')
plt.show()

We see that, althought the boundaries are not clearly seperated legandary True and False , around 650 is like a natural break point. 

Let's fit a Logistic Regression  algorithm that is used to predict the probability of a categorical dependent variable. In logistic regression, the dependent variable is a binary variable that contains data coded as 1 (yes, success, etc.) or 0 (no, failure, etc.). In our data legendary is a categorical dependent variable.

We need to reduce the categorical features for a better modelling.

In [ ]:
NewData = data.copy()
NewData['Legendary'].replace([True,False],[1,0],inplace=True)
NewData['Legendary'].tail(10)

In [ ]:
#we'll build a model with the data that categorical features droped.
#We select multiple features

features = ['HP', 'Attack', 'Defense', 
                    'Special_Attack', 'Special_Defense','Generation','Legendary']
LegModData = NewData[features].reindex()
LegModData.head()

In [ ]:
#LegModData['Legendary'] = LegModData_target

decisiontree = DecisionTreeClassifier() # defining  new object
train = LegModData[50:]   # seperated the first 50 rows are as test rest as train dataset
test = LegModData[:50]

x_train = LegModData.drop('Legendary', axis=1) # x_train as legendary state droped
y_train = LegModData['Legendary']  # y_train is the legendary value

x_test = test.drop('Legendary', axis=1) # same thing for test dataset
y_test = test['Legendary']

decisiontree.fit(x_train, y_train) # model decisiontree fit to x_train, y_train values

pred = decisiontree.predict(x_test) # make prediction to the model about x_test and keep in pred 

print("accuracy:", accuracy_score(y_test, pred)) #to test the accuracy of the model

**CONCLUTION**
>
>It is a fun dataset to deal with. It has a lot to do with. It is possible to do these analyses for each specific type or generation separately. We see that different generations have different characteristics and values. Also, Type1 and Type2 is an important distinction to get different results.  Lastly, we can say that legendary pokemon have superiority over not legendary ones by power. 
>
>
>----------------------